In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id = '1J8brwnHgopc8k_il49FEAB_3BdQvdCAU'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('lib.zip')

In [4]:
ls

adc.json  lib.zip  sample_data/


In [0]:
import os
# フォルダ「data」が存在しない場合は作成する
data_dir = "./lib/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
    import zipfile

    save_path = "lib.zip"
    
    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall("lib")  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [0]:
id = '1zd_JQpXU6AEi_x1xeTp1ryM4G2k59LVE'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('hymenoptera_data.zip')

In [0]:
import os
# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
    import zipfile

    save_path = "hymenoptera_data.zip"
    
    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall("data")  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [0]:
ls

adc.json  data/  lib/  sample_data/


In [0]:
cd data

/content/data


In [0]:
ls

hymenoptera_data/


In [0]:
cd hymenoptera_data

/content/data/hymenoptera_data


In [0]:
ls

train/  val/


In [0]:
cd train

/content/data/hymenoptera_data/train


In [0]:
cd /content

/content


*モデルの適用*

In [10]:
%%time
! npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.0
added 35 packages from 21 contributors in 1.697s
     |████████████████████████████████| 686kB 4.8MB/s 
     |████████████████████████████████| 204kB 49.8MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=8671ae469925ae63a8f6e990d7f213f452ec10d855ffc5ace6485fb9763a1711
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=8c32c86415c5c8f0b3f426d4f424d3e970aa7a5ac5c5ae6b964c92bf8bf5ea37
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
CPU times: user 97.1 ms, sys: 36.1 ms, total: 133 ms
Wall time: 16.7 s


In [11]:
%%time
import time
time.sleep(5)
! cat url.txt

CPU times: user 10.7 ms, sys: 6.47 ms, total: 17.2 ms
Wall time: 6.3 s


In [0]:
from lib.lib import network_finetuning as network
from lib.lib import architecture
from lib.lib import dataloader as dl
from lib.lib import optimizer
from lib.lib import trainer

import torchvision

In [13]:
%%time
net = network.InceptionV3(class_size=2)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [14]:
%%time
batch_size = 128
train_dataset = torchvision.datasets.ImageFolder(root='./data/hymenoptera_data/train', transform=dl.simple_transform(resize=299))
test_dataset = torchvision.datasets.ImageFolder(root='./data/hymenoptera_data/val', transform=dl.simple_transform(resize=299))
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size)

CPU times: user 1.34 ms, sys: 990 µs, total: 2.33 ms
Wall time: 2.6 ms


In [15]:
%%time
model = architecture.CNN_Architecture(net)

CPU times: user 5.48 ms, sys: 34 ms, total: 39.5 ms
Wall time: 69.1 ms


In [16]:
%%time
model.train(data_loader, epoch_count=50, is_inception=True)

cuda


Setting up a new session...


----------
Epoch 1/50 2020-01-23 11:29:46.073147
val Loss: 60662407.3203 Acc: 0.4575
----------
Epoch 2/50 2020-01-23 11:29:48.293144
train Loss: 1.7086 Acc: 0.2090
val Loss: 220924.7271 Acc: 0.4575
----------
Epoch 3/50 2020-01-23 11:29:55.449169
train Loss: 1.2056 Acc: 0.4385
val Loss: 2953.0758 Acc: 0.4575
----------
Epoch 4/50 2020-01-23 11:30:02.434852
train Loss: 0.9180 Acc: 0.5205
val Loss: 91.2798 Acc: 0.5425
----------
Epoch 5/50 2020-01-23 11:30:09.391410
train Loss: 0.8064 Acc: 0.8238
val Loss: 61.0279 Acc: 0.4575
----------
Epoch 6/50 2020-01-23 11:30:16.401895
train Loss: 0.7714 Acc: 0.5779
val Loss: 10.6540 Acc: 0.4575
----------
Epoch 7/50 2020-01-23 11:30:23.399366
train Loss: 0.7283 Acc: 0.6352
val Loss: 1.9230 Acc: 0.5425
----------
Epoch 8/50 2020-01-23 11:30:30.487382
train Loss: 0.6705 Acc: 0.8648
val Loss: 0.8013 Acc: 0.4379
----------
Epoch 9/50 2020-01-23 11:30:37.527295
train Loss: 0.6149 Acc: 0.9631
val Loss: 1.2813 Acc: 0.4575
----------
Epoch 10/50 2020-01-2

In [17]:
%%time
model.predict(data_loader['val'])

検証画像に対しての正解率： 54 %
CPU times: user 368 ms, sys: 292 ms, total: 660 ms
Wall time: 2.09 s
